In [43]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [44]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [45]:
model.children

<bound method Module.children of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [46]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [47]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [48]:
outputs

[tensor([[[[0.0000, 0.8773, 0.6040,  ..., 1.2178, 0.6019, 0.4184],
           [0.6238, 0.8079, 0.3892,  ..., 0.2831, 1.4299, 0.5385],
           [0.3839, 1.1379, 0.8157,  ..., 0.8474, 1.4296, 0.9341],
           ...,
           [1.5865, 0.3798, 0.9836,  ..., 0.6331, 1.1412, 0.8430],
           [0.5667, 1.6489, 0.3445,  ..., 0.3434, 0.0000, 0.6171],
           [0.6682, 0.1625, 0.4663,  ..., 0.1688, 0.2301, 1.1655]],
 
          [[0.5426, 0.3175, 0.1256,  ..., 0.6199, 0.5184, 0.8617],
           [0.7244, 0.0139, 0.6973,  ..., 0.8469, 1.1475, 0.8439],
           [0.6428, 0.6692, 0.9845,  ..., 0.6083, 0.4133, 0.7833],
           ...,
           [0.9645, 1.1162, 0.3883,  ..., 0.0558, 0.3073, 0.1116],
           [0.6230, 0.3062, 1.1175,  ..., 0.0000, 1.0787, 0.6970],
           [0.5461, 0.6939, 0.3043,  ..., 0.8491, 0.8783, 0.3080]],
 
          [[0.0000, 0.2090, 0.7490,  ..., 0.4120, 0.9551, 0.0740],
           [0.7981, 1.4180, 1.7061,  ..., 0.3336, 1.3403, 0.0000],
           [0.9891, 0.62

In [49]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [50]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                a=0,
                                mode='fan_out',
                                nonlinearity='relu')
        if m.bias is not None:
            init.ones_(m.bias)
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [51]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


